In [1]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging

In [2]:
#load all of our documents
with open('NHlegtext09_20.corpus','rb') as f:
    legcorpus = pickle.load(f)
    
#print a random doc
#print(legcorpus[303])

#assemble the text data, and remove the part where it says if it passes or not
combined_docs = {} #{bill_id:text}
total = 0
passed = 0

#a more unorthodox method of grabbing the target passage
leg_target = {} #{bill: 0 or 1, where 1 means pass}

#for each document, check if it is approved, record that, then combine the list into a string
for doc in legcorpus:
    pass_flag = 0
    if doc[0] not in combined_docs:
        total += 1
        try:
            if doc[2][-5] == 'approved':
                doc[2] = doc[2][:-5]
                passed += 1
                pass_flag = 1
                combined_docs[doc[0]] = " ".join(doc[2])
            else:
                combined_docs[doc[0]] = " ".join(doc[2])
                
        except:
            combined_docs[doc[0]] = " ".join(doc[2])
            pass
                
    leg_target[doc[0]] = pass_flag
    #for now, ignoring the amendments, but it seems like it could do better with the amendments
    #that would go here (i.e. an else statement saying doc[0] in combined_docs)
    
                   
print("\nTotal bills:", total)
print("Passed bills:", passed)
print("Bill passed ratio:", passed/total)


Total bills: 9520
Passed bills: 6
Bill passed ratio: 0.0006302521008403362


In [3]:
#actually, lets just stick with the given passages for now
leg_data = pd.read_csv("NHLegOverallData09_20.csv") #need all 10000, this is just 3000

with open('NHStateTargetList09_20.list','rb') as f:
    event_target = pickle.load(f)

with open('NHWinList09_20.list','rb') as f:
    win_state = pickle.load(f)
    
    
leg_targ = pd.DataFrame({"every stage passed":event_target}) #need all 10000, this is just 3000
#leg_targ = #append bill_id

bill_order = []
bill_target = []

leg_targ_old = leg_target
leg_target = {}

for i, bill_id in enumerate(leg_data["bill_id"]):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_targ["every stage passed"][i])
        leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
# bill_texts = []
# for bill_id in bill_order:
#     if bill_id in combined_docs:
#         bill_texts.append(combined_docs[bill_id])
#     else:
#         bill_texts.append("")
# passed = 0
# total = 0
# for i in leg_target:
#     passed+=leg_target[i]
#     total += 1
# print(passed/total)
#print(leg_target)

In [4]:
print(leg_targ[2100:2120])

                                     every stage passed
2100  minority committee report committee report oug...
2101            inexpedient to legislate motion adopted
2102            inexpedient to legislate motion adopted
2103                                     public hearing
2104                                  executive session
2105     committee report inexpedient to legislate vote
2106  inexpedient to legislate motion adopted voice ...
2107  inexpedient to legislate motion adopted voice ...
2108                                     public hearing
2109                                  executive session
2110                committee report ought to pass vote
2111                               removed from consent
2112                        lay on table motion adopted
2113                                                   
2114                                                   
2115                                     public hearing
2116                                            

In [4]:
#features are in a dictionary. 
#with open('BillFeatures.dict','rb') as f:
#    features = pickle.load(f)
features = leg_data
    
X = pd.DataFrame(features)
#print(X['committee introduced'].head())

# #make a mask so we only keep the rows that we actually have text for
# X_mask = [False]*len(X)
# for i, bill_id in enumerate(combined_docs): 
#     try:
#         print(X['bill_id'][X['bill_id'] == bill_id].index)
#         X['bill_id'][X['bill_id'] == bill_id].index[0]
#         X_mask[X['bill_id'][X['bill_id'] == bill_id].index[0]] = True
#     except:
#         print("i:",i,"bill_id:",bill_id)
#         raise IndexError 


# #next step: sift X so that it only contains ones we have docs for
# X_mask = pd.Series(X_mask)
# print(X_mask.iloc[4:9])
# X = X[X_mask]

#X = X.astype({'bill_id': 'int64'}).sort_values('bill_id', ascending=True, ignore_index=True)
#temporarily affix the target
X['target'] = leg_targ
X = X.astype({'year': 'int64'}).sort_values('year', ascending=True, ignore_index=True)
#date = #make date introduced something like 2018.50, so that we can sort numerically by month too
X = X.set_index('bill_id')

#print(X.loc[894687]['title'])
X.head()

Unnamed: 0  numeric_date  time_t  \
bill_id                                     
842619        16291          -1.0       1   
667269         7809          -1.0       3   
327994        83685          -1.0       2   
327994        83684          -1.0       1   
842619        16292          -1.0       2   

                                                     event  sponsor_vec  \
bill_id                                                                   
842619                                           committee          0.5   
667269   inexpedient to legislate motion adopted voice ...          1.0   
327994             ought to pass motion adopted voice vote          1.0   
327994                           motion adopted voice vote          1.0   
842619               referred to commerce consumer affairs          0.5   

         session  year chamber bill_type  \
bill_id                                    
842619      1201    -1       H         B   
667269      1141    -1       S         B   
327994        92    -1     NaN         R   
327994        92    -1     NaN         R   
842619      1201    -1       H         B   

                                                     title  ... primary_id  \
bill_id                                                     ...              
842619   Relative to abuse-deterrent opioid drug products.  ...      14488   
667269   Relative to training of certain board officers...  ...       4775   
327994   Requesting an opinion of the justices concerni...  ...       4760   
327994   Requesting an opinion of the justices concerni...  ...       4760   
842619   Relative to abuse-deterrent opioid drug products.  ...      14488   

        primary_party number_sponsors committee_introduced  month_introduced  \
bill_id                                                                        
842619              D               4                 None                -1   
667269              R               4                 None                -1   
327994              R               3                 None                -1   
327994              R               3                 None                -1   
842619              D               4                 None                -1   

        primary_leadership  primary_ideology  public_sentiment  press  \
bill_id                                                                 
842619                0.27               0.5            0.0000      0   
667269                0.27               0.5            0.1099      1   
327994                0.27               0.5            0.0000      0   
327994                0.27               0.5            0.0000      0   
842619                0.27               0.5            0.0000      0   

                                                    target  
bill_id                                                     
842619               referred to commerce consumer affairs  
667269   inexpedient to legislate motion adopted voice ...  
327994             ought to pass motion adopted voice vote  
327994             ought to pass motion adopted voice vote  
842619                                      public hearing  

[5 rows x 22 columns]

In [11]:
len(X)

124345

In [5]:
#Change this so it isn't so memory intensive in the future: combine the text later

#align the documents with their row to make one complete dataset
ordered_docs = [""]*len(X)

#align the targets with their row
ordered_target = [0]*len(X)
#print(leg_target)

#take the bill ids from the dataframe (which are in order)
ordered_bill_ids = list(X.index.values)
for i, b_id in enumerate(ordered_bill_ids):
    try:
        ordered_docs[i] = combined_docs[b_id]
    except:
        ordered_docs[i] = ""
        
    ordered_target[i] = leg_target[str(b_id)]
    if str(b_id) == '894761':
        print(i)
        print(ordered_target[i])
    

#make a new column for text    
X['legtext'] = ordered_docs

87974
public hearing
87975
public hearing
87976
public hearing
87977
public hearing
87978
public hearing
87979
public hearing
87980
public hearing
87981
public hearing
87983
public hearing
87984
public hearing
87985
public hearing
87986
public hearing
87992
public hearing


In [6]:
#list(X.index.values)[0:10]
ordered_target[0]

'referred to commerce consumer affairs'

In [7]:
list(combined_docs.keys())[0:10]
combined_docs[576685]
#ordered_target
X['legtext'].head()

bill_id
842619    act relative abusedeterrent opioid drug produc...
667269    act relative training certain board officers n...
327994    resolution requesting opinion justices concern...
327994    resolution requesting opinion justices concern...
842619    act relative abusedeterrent opioid drug produc...
Name: legtext, dtype: object

In [8]:
#create the entries for the data for demo
new_test_data = {'bill_id':[],
                 'time_t':[],
                 'event':[],
                 'sponsor_vec':[],
                 'chamber':[],
                 'bill_type':[],
                 'proposed_chamber':[],
                 'primary_name':[],
                 'primary_party':[],
                 'number_sponsors':[],
                 'committee_introduced':[],
                 'month_introduced':[],}            

#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

In [9]:
#select the data that we actually want to train on
#it is probably better to use primary id in the future instead of primary name
#X_new = X[['house/sentate', 'primary name', 'primary party', 'number of co sponsors', 'committee introduced','month introduced']].copy() #, 'committee introduced'
X_new = X[['time_t','event', 'sponsor_vec', 'chamber', 'bill_type', 'proposed_chamber', 'primary_name','primary_party','number_sponsors','committee_introduced','month_introduced']].copy()                      
#print(ordered_target[0:5])

amount_train = 90000
amount_test = 10000

X_dum = pd.get_dummies(X_new)

#can either manually define the features, or uncomment the below feature selection
#X_dum = X_dum[['number of co sponsors', 'house/sentate_S', 'month introduced_01', 'month introduced_05', 'month introduced_06', 'month introduced_12', 'primary party_R']]
X_dum_2 = X_dum[['time_t',                                                                            
'sponsor_vec',                                                                       
'number_sponsors',                                                                   
#'month_introduced',                                                                 
# 'event_committee report inexpedient to legislate vote',                              
# 'event_committee report ought to pass',                                              
# 'event_committee report ought to pass vote',                                         
# 'event_enrolled',                                                                    
# 'event_executive session',                                                           
# 'event_inexpedient to legislate motion adopted',                                     
# 'event_inexpedient to legislate motion adopted voice vote',                          
# 'event_inexpedient to legislate motion adopted voice vote bill killed',              
# 'event_interim study motion adopted voice vote',                                     
# 'event_majority committee report committee report inexpedient to legislate vote',    
# 'event_ought to pass motion adopted voice vote',                                     
# 'event_ought to pass ought to pass with amendment motion adopted voice vote',        
# 'event_public hearing',                                                              
# 'event_retained bill subcommittee committee work session',                           
# 'event_retained in committee committee',                                             
# 'event_signed',                                                                      
'chamber_H',                                                                         
'chamber_S',                                                                         
'bill_type_B',                                                                       
'bill_type_CA',                                                                      
'bill_type_CR',                                                                      
'proposed_chamber_H',                                                                
'proposed_chamber_S',                                                                
'primary_name_Daniel Itse',                                                          
'primary_name_Nancy Stiles',                                                         
'committee_introduced_None']] 

X_dum = pd.concat([X_dum_2, pd.get_dummies(X['event'])], axis=1)

print(X_dum.iloc[0])

ordered_target = pd.get_dummies(X['target'])

X_train = X_dum.iloc[0:amount_train]#.iloc
y_train = ordered_target.iloc[0:amount_train]#pd.get_dummies()
X_test = X_dum.iloc[amount_train:amount_train+amount_test]#.iloc
y_test = ordered_target.iloc[amount_train:amount_train+amount_test]#pd.get_dummies()




# indices = SelectKBest(f_classif, k=30).fit(X_train, y_train).get_support(indices=True)
# print(X_train.iloc[0,indices])
# #print('number of co sponsors')
# k_select = SelectKBest(f_classif, k=30)
# X_train = k_select.fit_transform(X_train, y_train)
# X_test = k_select.transform(X_test)
# #indices = 

# print(sum(y_train)/len(y_train))
# print(sum(y_test)/len(y_test))
#print(X_train[0:10])
#print(X_test[0:10])
#X_train.head()

time_t                         1.0
sponsor_vec                    0.5
number_sponsors                4.0
chamber_H                      1.0
chamber_S                      0.0
                              ... 
work session                   0.0
work session cancelled         0.0
work session continued         0.0
work session public hearing    0.0
work session recessed          0.0
Name: 842619, Length: 696, dtype: float64


In [18]:
y_test.head()

accedes to  accedes to committee motion adopted voice vote  \
bill_id                                                                  
1031394  0           0                                               0   
1031394  0           0                                               0   
1031394  0           0                                               0   
1031394  0           0                                               0   
1031394  0           0                                               0   

         accedes to motion adopted voice vote  \
bill_id                                         
1031394                                     0   
1031394                                     0   
1031394                                     0   
1031394                                     0   
1031394                                     0   

         accedes to request for committee of conference committee  \
bill_id                                                             
1031394                                                  0          
1031394                                                  0          
1031394                                                  0          
1031394                                                  0          
1031394                                                  0          

         accedes to request for committee of conference committee motion adopted  \
bill_id                                                                            
1031394                                                  0                         
1031394                                                  0                         
1031394                                                  0                         
1031394                                                  0                         
1031394                                                  0                         

         accedes to request for committee of conference committee motion adopted voice vote  \
bill_id                                                                                       
1031394                                                  0                                    
1031394                                                  0                                    
1031394                                                  0                                    
1031394                                                  0                                    
1031394                                                  0                                    

         accedes to request for committee of conference committee motion adopted voice vote recessed  \
bill_id                                                                                                
1031394                                                  0                                             
1031394                                                  0                                             
1031394                                                  0                                             
1031394                                                  0                                             
1031394                                                  0                                             

         accedes to request for committee of conference committee recessed  \
bill_id                                                                      
1031394                                                  0                   
1031394                                                  0                   
1031394                                                  0                   
1031394                                                  0                   
1031394                                                  0                   

         amendment adopted  ...  \
bill_id                     ...   
1031394                  0  ...   
1031394                  0  .

In [10]:
# compressed_legtext = pd.DataFrame(X['legtext'].copy())
# print(len(compressed_legtext))
# compressed_legtext['target'] = ordered_target
# compressed_legtext = compressed_legtext.loc[~compressed_legtext.index.duplicated(keep='first')]
# #compressed_legtext.drop_duplicates(subset=compressed_legtext.index, keep='first')
# #compressed_legtext.drop_duplicates(keep='first') 
# print(len(compressed_legtext))
# compressed_legtext.head()

124345
10612


legtext  \
bill_id                                                      
842619   act relative abusedeterrent opioid drug produc...   
667269   act relative training certain board officers n...   
327994   resolution requesting opinion justices concern...   
124688                                                       
804171   act prohibiting town meeting materially changi...   

                                            target  
bill_id                                             
842619       referred to commerce consumer affairs  
667269   committee report inexpedient to legislate  
327994     ought to pass motion adopted voice vote  
124688                              public hearing  
804171                              public hearing

In [11]:
#list(compressed_legtext['legtext'][0:8000])[0:2]


In [13]:
#prepare the documents by using tfidf vectors:

# #from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
# stemmer = SnowballStemmer("english", ignore_stopwords=True)
# #adjust this stemmer so that it does more than one word in the tokens

# class StemmedCountVectorizer(CountVectorizer):
#     def build_analyzer(self):
#         analyzer = super(StemmedCountVectorizer, self).build_analyzer()
#         return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

# mini_train = int(amount_train/len(X))*len(compressed_legtext)
# mini_test = int(amount_test/len(X))*len(compressed_legtext)

# stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,2))
# stem_counts = stemmed_count_vect.fit_transform(list(compressed_legtext['legtext'][0:mini_train]))
# stem_counts_test = stemmed_count_vect.transform(compressed_legtext['legtext'][mini_train:mini_train+mini_test])

# tfidf = TfidfTransformer()
# tfidf_vectors = tfidf.fit_transform(stem_counts)
# tfidf_vectors_test = tfidf.transform(stem_counts_test)

# k_best_tfidf = SelectKBest(f_classif, k=1000)
# final_tfidf_features = k_best_tfidf.fit_transform(tfidf_vectors, compressed_legtext['target'])
# final_tfidf_features_test = k_best_tfidf.transform(tfidf_vectors_test)

In [ ]:
#try withouth the words
#try with the state to state
#train the earlier fit with the overall classifier, it doesnt work with state to state

In [ ]:
# #combine tfidf legtext and the other features:
# legtext_train = final_tfidf_features.todense()
# #legtext_train = pd.DataFrame(legtext_train)#*10

# legtext_test = final_tfidf_features_test.todense()
# #legtext_test = pd.DataFrame(legtext_test)


# X_train = np.concatenate([np.array(X_train),legtext_train], axis=1)
# X_test = np.concatenate([np.array(X_test),legtext_test], axis=1)
# #X_train.head()
# X_train[0]

In [15]:
def perf_metric(y_test, y_pred):
#Accuracy
    correct=0
    total=0
    acc=0
    for i in range(len(y_test)):
        for j in range(len(y_test.iloc[0])):
            if y_test.iloc[i,j] == y_pred[i][j]:
                correct+=y_pred[i][j]
        total+=1
    acc=correct/total*100
    #print(x,y,len(y_pred),len(y_test))
    #print('Accuracy:', acc)
    return acc

def perf_metric(y_test, y_pred):
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)
    y_arg_test = np.argwhere(y_test == 1)[:,1]
    print("y_arg_test[0:10] :",y_arg_test[0:10])
    print("len(y_arg_test): ",len(y_arg_test))
    y_arg_pred = np.argwhere(y_pred == 1)
    _pred = [-1]*len(y_arg_test)
    print("Doing got loop...")
    for i, prediction in enumerate(y_arg_pred):
         _pred[prediction[0]] = prediction[1]
    print("Done with for loop")
    y_arg_pred = _pred
    
    print("y_arg_pred[0:10] :",y_arg_pred[0:10])
    print("len(y_arg_pred): ",len(y_arg_pred))
    y_correct = y_arg_test == y_arg_pred
    acc = np.mean(y_correct)
    return acc

In [16]:
print(np.array(y_test.iloc[-8]))
#print(predicted[1001])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [17]:
#train the neural network a bunch
means = []
for i in range(1):
    print('Training...')
    clf = MLPClassifier(alpha=0.0001, random_state=i, max_iter=20)#MultinomialNB(alpha=0)##DecisionTreeClassifier()#MultinomialNB()##19 ##
    clf.fit(X_train, y_train)
    print('Predicting...')
    predicted = clf.predict(X_test)
    predicted_prob = clf.predict_proba(X_test)
    #print(predicted_prob[0])
    #train_prediction = clf.predict(X_train)
    print('Mean Calculation...')
    new_mean = perf_metric(y_test, predicted)
    #new_mean = np.mean(predicted == y_test)
    means.append(new_mean)
    print(new_mean)
    #print(np.mean(train_prediction == y_train))
    
mean=np.mean(np.array(means))
print("overall mean:", mean)


Training...


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Predicting...
Mean Calculation...
y_arg_test[0:10] : [285 285 283  65 120 325 181 181  65 276]
len(y_arg_test):  10000
Doing got loop...
Done with for loop
y_arg_pred[0:10] : [-1, -1, 181, -1, 120, 325, 181, 181, -1, -1]
len(y_arg_pred):  10000
0.4477
overall mean: 0.4477


In [18]:
print(X_train.shape)
print(y_train.shape)


(90000, 696)
(90000, 653)


In [28]:
#MNBclf = clf\
#MLPclf = clf
#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298
#prediction_vector = MNBclf.predict_proba(X_dum.loc[1287298])
#print(prediction_vector)

In [168]:
#save the state-to-state classifier
#StSclf = clf #this has been run and saved

In [19]:
#y_test
def vec_to_state(prediction_vectors, example_output_df):
    index = np.array(example_output_df.columns)
    print('index[0:10]',index[0:10])
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    prediction_vectors = prediction_vectors == 1
    predictions = []
    for prediction in prediction_vectors:
        #print(prediction)
        
        if len(index[prediction]) == 0:
            append_value = [0]
        else:
            append_value = index[prediction][0]
            
        print('append_value:', append_value)
        predictions.append(append_value)
        
    return predictions

def prob_vec_to_n_states(prediction_vector, example_output_df, n=1):
    #return the names, and probabilities in order
    index = np.array(example_output_df.columns)
    
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    ind = np.argpartition(prediction_vector, -n)[-n:]
    sorted_ind = ind[np.argsort(prediction_vector[ind])][::-1]
    
    predictions = []
    for index_s in sorted_ind:
        append_value = index[index_s]
            
        print('append_value:', append_value)
        predictions.append((append_value, prediction_vector[index_s]))
        
    return predictions
    


#pd.get_dummies(X['event'])
def state_to_vec(current_state, example_output_df):
    X_index = np.array(example_output_df.columns)
    indices = current_state #a string

    #for index in indices:
    location = X_index == index
    location = location.astype(int)
        #print(location)
    return(location)
    

In [20]:
pd.get_dummies(X[['event', 'month_introduced']]).columns

Index(['month_introduced', 'event_accedes to',
       'event_accedes to committee motion adopted voice vote',
       'event_accedes to motion adopted voice vote',
       'event_accedes to request for committee of conference committee',
       'event_accedes to request for committee of conference committee motion adopted',
       'event_accedes to request for committee of conference committee motion adopted voice vote',
       'event_accedes to request for committee of conference committee motion adopted voice vote recessed',
       'event_accedes to request for committee of conference committee recessed',
       'event_amendment adopted',
       ...
       'event_without objection vacated from public municipal affairs',
       'event_without objection vacated from public municipal affairs judiciary',
       'event_without objection vacated from public municipal affairs referred to finance',
       'event_without objection vote motion adopted',
       'event_without objection ways and m

In [21]:
#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298
StSclf = clf

#save the state to state data so we can make a prediction on it next

#see what is actually in there
#print(X_dum.loc[1294143])
#print(X_dum.loc[1287298])
print(X.loc[1287298])
prediction_vector = StSclf.predict_proba(X_dum.loc[1287298])[-1]#1294143])
#make an initial prediction of the next step
print(prob_vec_to_n_states(prediction_vector, y_test, n=10))

#predictions = vec_to_state(prediction_vectors, y_test)
#print(predictions)
#X_dum.head()

         Unnamed: 0  numeric_date  time_t  \
bill_id                                     
1287298      119847   2019.919355       1   
1287298      119848   2019.919355       2   
1287298      119849   2019.919355       3   
1287298      119850   2019.919355       4   
1287298      119851   2019.919355       5   

                                                     event  sponsor_vec  \
bill_id                                                                   
1287298  introduced referred to environment and agricul...          0.0   
1287298                            recessed public hearing          0.0   
1287298                           continued public hearing          0.0   
1287298                             committee work session          0.0   
1287298                                  executive session          0.0   

         session  year chamber bill_type  \
bill_id                                    
1287298     1729  2019       H         B   
1287298     1729  2019    

In [144]:
#DTClf has been saved
#DTClf = clf

In [23]:

#list(X_dum.keys()[indices])


In [ ]:
# #only run this if using MultinomialNB classifier
# feature_indices = indices
# pass_probs = clf.feature_log_prob_[1]
# feature_pass_probs = {feature:pass_probs[i] for i, feature in enumerate(feature_indices)}

# sorted_features = sorted(feature_indices, key=lambda x: feature_pass_probs[x], reverse=True)
# #print(sorted_features)

# vocab_dict = {indices[i]:X_dum.keys()[indices][i] for i, _ in enumerate(indices)}
# #print(vocab_dict)

# for i, feature in enumerate(sorted_features):
#     if i >= 1000:
#         break
#     print(vocab_dict[feature], "\t\t", feature_pass_probs[feature])

In [ ]:
# print(len(predicted))
# print()
# for i, prediction in enumerate(predicted):
#     if i == 50:
#         break
#     print("real:", y_test[i],"\tpredicted:", predicted_prob[i][1])#, "\tdata:", X_dum.iloc[amount_train:amount_train+amount_test,indices])
